In [2]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset, Dataset, concatenate_datasets
import transformers

persian_dataset = load_dataset("AliFartout/PEYMA-ARMAN-Mixed")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.21k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26417 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3302 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3303 [00:00<?, ? examples/s]

In [12]:
persian_dataset['train'].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'tokens': Value(dtype='string', id=None),
 'ner_tags': Value(dtype='string', id=None),
 'ner_tags_names': Value(dtype='string', id=None)}

In [13]:
tags = ['B_DAT', 'B_LOC', 'B_MON', 'B_ORG', 'B_PCT', 'B_PER', 'B_TIM', 'I_DAT',
        'I_LOC', 'I_MON', 'I_ORG', 'I_PCT', 'I_PER', 'I_TIM', 'B_LOC', 'I_LOC',
        'B_PER', 'I_PER', 'B_ORG', 'I_ORG', 'B_FAC', 'I_FAC', 'B_EVE', 'I_EVE',
        'B_PRO', 'I_PRO', 'O']

unique_tags = list(dict.fromkeys(tags))

tag_to_int = {tag: idx for idx, tag in enumerate(unique_tags)}

print(tag_to_int)


{'B_DAT': 0, 'B_LOC': 1, 'B_MON': 2, 'B_ORG': 3, 'B_PCT': 4, 'B_PER': 5, 'B_TIM': 6, 'I_DAT': 7, 'I_LOC': 8, 'I_MON': 9, 'I_ORG': 10, 'I_PCT': 11, 'I_PER': 12, 'I_TIM': 13, 'B_FAC': 14, 'I_FAC': 15, 'B_EVE': 16, 'I_EVE': 17, 'B_PRO': 18, 'I_PRO': 19, 'O': 20}


In [14]:
import re

persian_dataset

def formalization(example):


    literals = re.findall(r"'\s*([^']+?)\s*'", example['ner_tags_names'])
    example['ner_tags_names'] = literals
    example['ner_tags'] = [tag_to_int[tag] for tag in literals]
    example['tokens'] = re.findall(r"'\s*([^']+?)\s*'", example['tokens'])

    return example

persian_aligned = persian_dataset.map(formalization)


Map:   0%|          | 0/26417 [00:00<?, ? examples/s]

Map:   0%|          | 0/3302 [00:00<?, ? examples/s]

Map:   0%|          | 0/3303 [00:00<?, ? examples/s]

In [15]:
persian_aligned

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'tokens', 'ner_tags', 'ner_tags_names'],
        num_rows: 26417
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'tokens', 'ner_tags', 'ner_tags_names'],
        num_rows: 3302
    })
    test: Dataset({
        features: ['Unnamed: 0', 'tokens', 'ner_tags', 'ner_tags_names'],
        num_rows: 3303
    })
})

In [ ]:
def check_lengths(example):
    return len(example['tokens']) == len(example['ner_tags']) == len(example['ner_tags_names'])

for split in ['train', 'validation', 'test']:
    correct_rows = persian_aligned[split].filter(check_lengths)

    persian_aligned[split] = correct_rows

    print(f"Rows in {split} after removing mismatches: {persian_aligned[split].num_rows}")

In [31]:
def check_lengths(example):
    if len(example['tokens']) == len(example['ner_tags']) == len(example['ner_tags_names']):
        return True
    return False

for split in ['train', 'validation', 'test']:
    mismatches = persian_aligned[split].filter(lambda x: not check_lengths(x))

    if mismatches.num_rows > 0:
        print(f"Issues found in {split} split:")
        print(mismatches)
    else:
        print(f"All rows in {split} have matching lengths.")


Filter:   0%|          | 0/26384 [00:00<?, ? examples/s]

All rows in train have matching lengths.


Filter:   0%|          | 0/3296 [00:00<?, ? examples/s]

All rows in validation have matching lengths.


Filter:   0%|          | 0/3296 [00:00<?, ? examples/s]

All rows in test have matching lengths.


In [40]:

persian_aligned = persian_aligned.remove_columns('Unnamed: 0')

In [41]:
from datasets import DatasetDict , Sequence , Value , Features , ClassLabel
tags = list(set(tags))
ner_tags = ClassLabel(num_classes=len(tags) , names=tags)

persian_aligned = persian_aligned.cast_column('ner_tags',Sequence(ner_tags))

Casting the dataset:   0%|          | 0/26384 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3296 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3296 [00:00<?, ? examples/s]

In [42]:
persian_aligned['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['B_LOC', 'I_DAT', 'B_PCT', 'I_LOC', 'I_PER', 'I_MON', 'B_ORG', 'B_PRO', 'B_PER', 'O', 'I_PCT', 'I_ORG', 'B_FAC', 'B_DAT', 'B_TIM', 'I_TIM', 'I_EVE', 'B_MON', 'I_PRO', 'B_EVE', 'I_FAC'], id=None), length=-1, id=None),
 'ner_tags_names': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [43]:
persian_aligned['train'][0]

{'tokens': ['یوکوویچ',
  'متولد',
  'دانمارک',
  'است',
  'اما',
  'والدین',
  'او',
  'صرب',
  'هستند',
  '.'],
 'ner_tags': [5, 20, 1, 20, 20, 20, 20, 1, 20, 20],
 'ner_tags_names': ['B_PER',
  'O',
  'B_LOC',
  'O',
  'O',
  'O',
  'O',
  'B_LOC',
  'O',
  'O']}